## Connection and Data Validation Notebook

## Table of Contents
* [Check for Training Data in Project Space](#DataCheck)
    * [Load the Training Data from Db2 if it does not exist in the project space](#section_1_1)
    
    * [Check the connection and data loading](#section_1_2)
  
* [Data Validation](#chapter2)
    * [Split the Data](#Optional)

    * [Generate Training Stats on both Splits](#section_2_2)
    * [Infer Schema on both Splits](#section_2_3) 
    * [Check for anomalies](#section_2_4) 
    * [Return a boolean to validate the tests](#section_3_1) 


## Imports

In [ ]:
!pip install tensorflow-data-validation

In [ ]:
!python3 --version

In [ ]:
# this is a workaround as long as there are issues with custom python enviroments in multiple projects
!pip install ibm_watson_studio_pipelines

In [4]:
from botocore.client import Config
from sklearn.model_selection import train_test_split
from dataclasses import dataclass

import itc_utils.flight_service as itcfs


# TODO: Await (or build) for py3.10 version
import tensorflow_data_validation as tfdv
import numpy as np
import pandas as pd

from ibm_watson_studio_pipelines import WSPipelines

import logging
import os, types
import warnings
import requests

warnings.filterwarnings("ignore")

2023-10-23 07:24:33.933101: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-23 07:24:33.933161: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-23 07:24:33.933204: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-10-23 07:24:35.469552: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


### Load the Credentials


These environment variables are set in WS Pipelines

In [5]:
TOKEN = os.getenv("USER_ACCESS_TOKEN")

In [7]:
training_file_name = "german_credit_data_biased_training.csv"
mlops_path = "mlops-dir"

In [ ]:
if os.path.exists(mlops_path):
    print("Path exists")
else:
    # create path
    os.mkdir(mlops_path)
    # download training data csv from https://raw.githubusercontent.com/IBM/monitor-wml-model-with-watson-openscale/master/data/german_credit_data_biased_training.csv
    url = "https://raw.githubusercontent.com/IBM/monitor-wml-model-with-watson-openscale/master/data/german_credit_data_biased_training.csv"
    response = requests.get(url)

    # Check if the request was successful
    if response.status_code == 200:
        with open("german_credit_data_biased_training.csv", "wb") as file:
            file.write(response.content)
        print("Downloaded and saved as 'german_credit_data_biased_training.csv'")
    else:
        print("Failed to download the CSV file. Status code:", response.status_code)



In [ ]:
training_file_path = os.path.join(mlops_path, training_file_name)

## Check for Training Data in Project Space

In [13]:
def check_for_file_in_filesystem(path):
    if os.path.exists(path):
        return True
    else:
        return False
    
def read_data_from_db2(data_request):
    read_client = itcfs.get_flight_client()
    DB2_DATA_data_request = {
        'connection_name': """DB2_DATA""",
        'interaction_properties': {
            'select_statement': 'SELECT * FROM "CUSTOMER_DATA"."GERMAN_CREDIT_RISK_TRAINING" FETCH FIRST 5000 ROWS ONLY'
        }
    }

    flightInfo = itcfs.get_flight_info(read_client, nb_data_request=data_request)

    df = itcfs.read_pandas_and_concat(read_client, flightInfo, timeout=240)
    return df
    
def load_data_from_project(path):
    body = check_for_file_in_filesystem(path)
    if body:
        gcf_df = pd.read_csv(path)
        return gcf_df
    else:
        print("\n")
        print(f"{path} file/path is probably not in project. Loading File from MLOps COS Bucket.")

        data_request = {
                'connection_name': """DB2_DATA""",
                'interaction_properties': {
                    'select_statement': 'SELECT * FROM "CUSTOMER_DATA"."GERMAN_CREDIT_RISK_TRAINING" FETCH FIRST 5000 ROWS ONLY'
                }
            }

        gcf_df = read_data_from_db2(data_request)
        return gcf_df

## Load the Training Data from Db2 if the file doesn't exist

In [14]:
gcr_df = load_data_from_project(training_file_path)

## Encode for ease of use with OpenScale
gcr_df['Risk'] = gcr_df['Risk'].map({'Risk':1,'No Risk':0})
gcr_df.head()

NameError: name 'training_file_path' is not defined

## Data Validation 

In [ ]:
@dataclass
class Datavalidation:
    """
    
    Data Validation Class
    
    """
    dataframe : pd.DataFrame
    mask_per :int
    
    
    def split_data(self,seed=32):
        """
        Split Data into Train and Test Splits
        
        """
        np.random.seed(seed)
        mask = np.random.rand(len(self.dataframe)) <= self.mask_per
        training_data = gcr_df[mask]
        testing_data = gcr_df[~mask]

        print(f"No. of training examples: {training_data.shape[0]}")
        print(f"No. of testing examples: {testing_data.shape[0]}")
        
        return training_data, testing_data
    
    # TODO: Replace with Db2/fileystem
    def save_data_in_filesystem(self,df,filename):
        """
        Save Data in Filesystem

        Passed filename should involve path

        """
        try:
            df.to_csv(filename,index=False)
            print(f"File {filename} persisted successfully")
        except Exception as e:
            print(e)
            print(f"File serialization for {filename} failed")
    
    def generate_statistics(self,df):
        """
        
        Generate Statistics on a given Dataframe
        
        """
        train_stats = tfdv.generate_statistics_from_dataframe(df)
        tfdv.visualize_statistics(train_stats)
        return train_stats
    
    def inferSchema(self,stats):
        
        """
        InferSchema on a given Dataframe
        
        """
        schema = tfdv.infer_schema(statistics=stats)
        tfdv.display_schema(schema=schema)
        return schema
    
    def compare_statistics(self,lhs,rhs):
        """
        
        Compare Statistics between a test dataframe and reference Schema
        
        """
        # Compare evaluation data with training data
        tfdv.visualize_statistics(lhs_statistics=lhs, rhs_statistics=rhs,
                                  lhs_name='TEST_DATASET', rhs_name='TRAIN_DATASET')
        
        
    def check_for_anomalies(self,testable_stats,ref_schema):
        """
        
        Check for any anomalies based on statistics and schema and values
        
        """
        anomalies = tfdv.validate_statistics(statistics=testable_stats, schema=ref_schema)
        tfdv.display_anomalies(anomalies)
        if len(anomalies.anomaly_info.items()) > 0:
            logger.error("Anomalies found in dataset...")
            logger.error(str(self.anomalies.anomaly_info.items()))
            return True
        else:
            return False

###  Split Data into Train and Eval Splits to Check for Consistency

In [ ]:
classvalidate = Datavalidation(dataframe=gcr_df,mask_per=0.8) 

training_data, testing_data = classvalidate.split_data()

## Generate Training Stats on both Splits

In [ ]:
train_stats = classvalidate.generate_statistics(training_data)

In [ ]:
test_stats = classvalidate.generate_statistics(testing_data)

## Infer Training Data Schema

In [ ]:
train_schema = classvalidate.inferSchema(train_stats)

## Infer Test Data Schema

In [ ]:
test_schema = classvalidate.inferSchema(test_stats)

## Compare Eval and Train Data 

In [ ]:
classvalidate.compare_statistics(lhs=test_stats,rhs=train_stats)

## Check For Data Anomalies 

### Check eval data for errors by validating the eval data stats using the previously inferred schema.

In [ ]:
anomaly_status = classvalidate.check_for_anomalies(test_stats,train_schema)
anomaly_status

## Save Train and Test Data for Data Preparation Stage

In [ ]:
train_data_filename = "train_gcr.csv"
test_data_filename = "test_gcr.csv"
train_data_path = os.path.join(path, train_data_filename)
test_data_path = os.path.join(path, test_data_filename)

In [ ]:
anomaly_status = False

In [ ]:
# TODO: Replace with Db2/fileystem
if not anomaly_status:
    classvalidate.save_data_in_filesystem(df=training_data,filename=train_data_path)
    classvalidate.save_data_in_filesystem(df=testing_data,filename=test_data_path)

## Check if files Exists in COS

In [ ]:
# TODO: Replace with Db2/fileystem
files_copied_in_cos = check_for_file_in_filesystem(train_data_path) and check_for_file_in_filesystem(test_data_path)
files_copied_in_cos

## Register a Boolean Variable in WS Pipeline

In [ ]:
validation_params = {}
validation_params['anomaly_status'] = anomaly_status
validation_params['files_copied_in_cos'] = files_copied_in_cos
validation_params['train_data_filename'] = train_data_filename
validation_params['test_data_filename'] = test_data_filename

In [ ]:
# pipelines_client = WSPipelines.from_token(token=TOKEN)
pipelines_client = WSPipelines.from_token(TOKEN)
pipelines_client.store_results(validation_params)